# Practice: Spotify's API @ ESADE

This notebook contains a set of exercises that will guide you through the different steps of this practice exercise. Devise solutions that are code-based, i.e. not hard-coded or manually computed. You can monitor your progress by running the tests for each exercise. Thsi will allow you to spot your mistakes and correct them.

<div class="alert alert-success">The aim of this exercise is to create and save a dataset containing information about every song in a given playlist by requesting data from Spotify's API.</div>

## Getting client credentials

Spotify's API uses OAuth as an Authentication scheme. Hence, before starting to make requests, you need to get your client credentials to the Spotify API. 

To do so, you need to have a Spotify account (free or paid). If you don't have one yet, please create a free account before moving on. Once you do, head over to Spotify for Developers, open your [Dashboard](https://developer.spotify.com/dashboard/) and log in with your account. 

![Dashboard](https://www.dropbox.com/s/cpfepk5fbq6ic5a/dashboard.png?raw=1)

Click on “CREATE AN APP”, choose a name and description for your project and work your way through the checkboxes. 

<img src="https://www.dropbox.com/s/afubgs4ar99uh80/app.png?raw=1" width="300">

Don't worry about the actual name and description. The only thing we are interested in is getting the credentials.

![Credentials](https://www.dropbox.com/s/3mmxxeet61nha4l/credentials.png?raw=1)

Once your App has been created, you should see a “Client ID” and “Client Secret” on the left-hand side. These numbers correspond to your client credentials.

<div class="alert alert-info">Create two new variables, <b>client_id</b> and <b>client_secret</b>, that store your ID and Key, respectively</div>

In [276]:
client_id  = "99db58f8b86b47b495b4748d2bae49ae"
client_secret = "010d0d26d46140e592f1d42811c15115"

These are the keys that you will use throughout the remaining of the assignment.

Run the following cell to check that you created them properly. If you get no error when running the cell it means that everything is OK. You can run this cell as many times as you want as long as you **don't modify it**.

<div class="alert alert-danger"> Make sure you pass this test successfully before moving on to the remaining exercises.</div>

In [277]:
#Revert and understand the unittest dictionary and the test code pasted here

In [278]:
import unittest

tc = unittest.TestCase()
tc.assertTrue('client_id' in locals())
tc.assertTrue('client_secret' in locals())
tc.assertIsInstance(client_id, str)
tc.assertIsInstance(client_secret, str)
tc.assertTrue(client_id.isalnum())
tc.assertTrue(client_secret.isalnum())

Great! We are good to go. Next step is getting an access token.

## Getting an access token

In order to access the various endpoints of the Spotify API, we need to pass an access token. To get one, we need to pass a ```POST``` request with our client credentials. This request will create a token resource in the server and respond back with it. We can build this ```POST``` request using ```requests``` library. 

<div class="alert alert-info">Run the following cell to built your POST request</div>

In [279]:
import requests

# URL for token resource
auth_url = 'https://accounts.spotify.com/api/token'

# request body
params = {'grant_type': 'client_credentials',
          'client_id': client_id,
          'client_secret': client_secret}

# POST the request
auth_response = requests.post(auth_url, params).json()

Take a look at the response you obtained. The access_token you just retrieved will expire after one hour.

<div class="alert alert-info">Retrieve your token from <b>auth_response</b> and save it in a new variable called <b>access_token</b>. </div>

<div class="alert alert-warning">Make sure you define the <i>access_token</i> variable such that it will be updated each time your code is run from scratch, i.e. make sure it hasn't expired by the time your code is graded.</div>

In [280]:
access_token = auth_response['access_token']
#print(auth_response)
#print(access_token)

As above, the access token will be used throughout the remaining of the assignment.

Run the following cell to check that you created it properly. If you get no error when running the cell it means that everything is OK. You can run this cell as many times as you want as long as you **don't modify it**.

<div class="alert alert-danger"> Make sure you pass this test successfully before moving on to the remaining exercises.</div>

In [281]:
import unittest

tc = unittest.TestCase()
tc.assertTrue('access_token' in locals())
tc.assertIsInstance(access_token, str)

This token is your golden ticket to access Spotify's API. A copy of this string is now stored in the server, so that everytime you make a request to the API the server will check that the token you provide and the one it has in store match.

<img src="https://www.dropbox.com/s/hgb02k4h1mtdv22/header.png?raw=1" width="500">

As opposed to NASA's API, where we provided our API Key as part of the request body, Spotify's API expects you to include your access token in the requests header. There is a specific header called 'Authorization' for this purpose. Providing this information is sometimes tricky. Hence, the header has already been formatted for you. 

<div class="alert alert-info">Run the following cell to save the header in a new variable so that you can use it later on.</div>

In [282]:
#Clarify how do we learn to do this?

In [283]:
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

## Poking around

Spotify's API provides numerous endpoints to access things like album listings, artist information, playlists, even Spotify-generated audio analysis of individual tracks, which include their time signature or measurements such as their “danceability” or "loudness". You can take a look at all the information available by reading the [Docs](https://developer.spotify.com/documentation/web-api/reference/). In this assignment you will use several of these endpoints.

In order to get a feel of how the API works, we will begin by making a ```GET``` request to the ```audio-features``` endpoint to extract data for a specific track. In particular, let's retrieve all the information for Radiohead's *Creep* song. 

The first thing you need is to identify the appropriate URL or path to direct your request to. The urls for all Spotify API endpoints follow the same structure. They all use the base URL for the API and are then defined as a concatenation of ```base_url + endpoint```. Sometimes, you will also need to provide some additional information as part of the URL. In the case of ```audio-features```, however, it is enough with just the ```base_url``` and the ```endpoint``` name.

The ```base_url``` is defined below:

In [284]:
base_url = 'https://api.spotify.com/v1/'

<div class="alert alert-info">Define the url for the audio-features endpoint by following the instructions above. Store it in a variable called <b>audio_features_endpoint</b>.</div>

In [285]:
audio_features_endpoint = 'audio-features'
url = base_url+audio_features_endpoint

Next thing we need is to fill in the request body. If you check the documentation you'll see that the ```audio-features``` endpoint takes the following query parameters.

<img src="https://www.dropbox.com/s/s4zs6wlue0u16cu/body.png?raw=1" width="500">

Hence, the final thing you need to extract data about Radiohead's Creep song is to locate its ```id```. This is its unique identifier. Spotify has unique ids for tracks, for artists, for albums, for playlists, etc.

![Creep](https://www.dropbox.com/s/kufj6ww2yn069gb/creep.png?raw=1)

You can get the ```id``` for any song by going to Spotify, looking for the song, clicking the “…” by the song name, then “Share” and then “Copy Spotify URI”. 

<div class="alert alert-warning">Note that this procedure also works for retrieving ids for artists, albums or any other resource type. Alternatively, you can also retrieve the id for a song or a playlist by clicking "..." by the name, then "Share" and the "Copy Link". This will return a full link to the resource. The id corresponds to the string right after the last slash sign / and before the ? sign</div>

This URI should be a string that includes something like **spotify:track:**, followed by an alphanumeric sequence. This sequence is the ID you are looking for.

<div class="alert alert-info">Create a new variable called <b>track_id</b> that stores the ID for Radiohead's song Creep.</div>

In [286]:
track_id = '70LcF31zb1H0PyJoS1Sx1r'

Now that you have the id, let's format the body of the request. As you did for NASA's API, you can provide the body in dictionary form. The keys of the dictionary should correspond to the different query parameters in the documentation.

<div class="alert alert-info">Create a dictionary called <b>params</b> that stores the body of your request. Make sure you format it in the right way.</div>

In [287]:
params = {'ids':track_id}

Now that everything is ready, you can send the actual GET request to retrieve the data.

<div class="alert alert-info">Write the code to make your get request using the requests library. When doing so, remember to pass the <i>url</i>, the <i>headers</i> and the <i>params</i> dictionary as arguments to the <i>get</i> function. Convert the response to <i>json</i> format and store it in a new variable called <b>creep</b>.</div>

In [288]:
import requests
creep = requests.get(url,params=params,headers=headers)
creep.json()

{'audio_features': [{'danceability': 0.515,
   'energy': 0.43,
   'key': 7,
   'loudness': -9.935,
   'mode': 1,
   'speechiness': 0.0372,
   'acousticness': 0.0097,
   'instrumentalness': 0.000133,
   'liveness': 0.129,
   'valence': 0.104,
   'tempo': 91.844,
   'type': 'audio_features',
   'id': '70LcF31zb1H0PyJoS1Sx1r',
   'uri': 'spotify:track:70LcF31zb1H0PyJoS1Sx1r',
   'track_href': 'https://api.spotify.com/v1/tracks/70LcF31zb1H0PyJoS1Sx1r',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/70LcF31zb1H0PyJoS1Sx1r',
   'duration_ms': 238640,
   'time_signature': 4}]}

You can run the following cell to check if you obtained the right answer. If you get no error when running the cell it means that you did right. Otherwise, revise your code to ensure you get no error. You can run this cell as many times as you want, just **remember not to modify it**.

<div class="alert alert-danger"> Make sure you pass this test successfully before moving on to the remaining exercises. </div>

<div class="alert alert-warning">Often Spotify has several instances of a track in its catalogue, each available in a different set of markets. This commonly happens when the track the album is on has been released multiple times under different licenses in different markets. These tracks are linked together so that when a user tries to play a track that isn’t available in their own market, the Spotify mobile, desktop, and web players try to play another instance of the track that is available in the user’s market. Even if this issue won't prevent you from being able to play the  song in your device, it might result in different ids for the different markets. Note that the test below is written taking the values for the Spanish market into account.</div>

<div class="alert alert-warning">Note that you can specify the market for this and for future queries through the <i>market</i> parameter.</div>

In [289]:
tc = unittest.TestCase()
tc.assertEqual(creep, {'audio_features': [{'danceability': 0.515,
                                         'energy': 0.43,
                                         'key': 7,
                                         'loudness': -9.935,
                                         'mode': 1,
                                         'speechiness': 0.0369,
                                         'acousticness': 0.0102,
                                         'instrumentalness': 0.000141,
                                         'liveness': 0.129,
                                         'valence': 0.104,
                                         'tempo': 91.841,
                                         'type': 'audio_features',
                                         'id': '6b2oQwSGFkzsMtQruIWm2p',
                                         'uri': 'spotify:track:6b2oQwSGFkzsMtQruIWm2p',
                                         'track_href': 'https://api.spotify.com/v1/tracks/6b2oQwSGFkzsMtQruIWm2p',
                                         'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6b2oQwSGFkzsMtQruIWm2p',
                                         'duration_ms': 238640,
                                         'time_signature': 4}]})

AssertionError: <Response [200]> != {'audio_features': [{'danceability': 0.51[494 chars] 4}]}

Congrats! You just made your first successful request to Spotify's API! Feel free to take a look at the information included in the response. Pay special attention to the way in which information is presented. Once you are done, let's move on to some actual work!

## Getting data from a playlist

In the following exercise you will build a dataset containing data about different songs. You can either use a playlist of your own, or use the one we have created for this purpose.

You can find our playlist in the following [link](https://open.spotify.com/playlist/4NVeFUEHBybfh3ITNG1b8n?si=js9BKt5aTOiCWMm_Cx4Vvg). 

<div class="alert alert-info">Create a variable called <b>playlist_id</b> that stores the id of your playlist of choice.</div>

<div class="alert alert-warning">Your <i>playlist_id</i> should contain only alphanumeric characters. This means that characters such as ? $ % & / ! should not be included.</div>

In [290]:
playlist_id = "4NVeFUEHBybfh3ITNG1b8n"

In the following you are going to extract information about the different tracks included in this playlist. Hence, let's make sure that the variabl ei spropelry created and that it refers to an actual id.

As above, run the cell below. If you get no error when running the cell it means that you did right. Otherwise, revise your code to ensure you get no error. You can run this cell as many times as you want, just **remember not to modify it**.

<div class="alert alert-danger"> Make sure you pass this test successfully before moving on to the remaining exercises.</div>

In [291]:
tc = unittest.TestCase()
tc.assertTrue('playlist_id' in locals())
tc.assertIsInstance(playlist_id, str)
tc.assertTrue(playlist_id.isalnum())
tc.assertTrue('spotify:track:' not in playlist_id)

The next step will be making a request to get full details of the tracks included in your chosen playlist. Remember that you can take a look at all the information available at the different endpoints in Spotify's API by reading the [Docs](https://developer.spotify.com/documentation/web-api/reference/). Locate the right endpoint for your query and read the Docs to find out how to build your request. 

<div class="alert alert-info"><b>Exercise 1 </b>Write the code to retrieve all the items from your chosen playlist from the <i>tracks</i> endpoint. When making your request don't use any of the optional arguments. Direct your request to the right endpoint instead. Store the <i>raw</i> response in a new variable called <i>playlist_response</i>.
</div>

<div class="alert alert-warning">To complete this exercise you must use the requests library</div>

In [292]:
playlisturl = base_url+"playlists/"+playlist_id+"/tracks"
playlist_response= requests.get(playlisturl,headers=headers)
playlist_response

<Response [200]>

The following cells contain the tests that will grade your code. **Don't modify them**. Simply leave them as they are.

In [293]:
# Variable playlist_response exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('playlist_response' in locals())
tc.assertIsInstance(playlist_response, requests.models.Response)

In [294]:
# Generate id if inexistent
if "playlist_id" not in locals():
    if not isinstance(playlist_response, dict):
        playlist_id = playlist_response.json()['id']
    else:
        playlist_id = playlist_response['id']
elif not playlist_id.isalnum():
    playlist_id = playlist_id.split('?')[0]

# Request has been directed to the right endpoint
tc = unittest.TestCase()
tc.assertEqual(playlist_response.url, 'https://api.spotify.com/v1/playlists/'+playlist_id+'/tracks')

In [295]:
# Response is a success
tc = unittest.TestCase()
tc.assertEqual(playlist_response.status_code, 200)

<div class="alert alert-info"><b>Exercise 2 </b>Convert the response to JSON format and store the result in a new variable called <i>playlist</i>.</div>

In [296]:
playlist = playlist_response.json()

The following cells contain the tests that will grade your code. **Don't modify them**. Simply leave them as they are.

In [297]:
# Variable playlist exists
tc = unittest.TestCase()
tc.assertTrue('playlist' in locals())

In [298]:
# Variable playlist is of correct type
tc = unittest.TestCase()
tc.assertIsInstance(playlist, dict)

Take your time to familiarize yourself with the data and how they are presented. Note that, by default, Spotify's API only returns information about a maximum of 100 tracks in a playlist. If your playlist of choice has more that 100 tracks, you'll retrieve the data only for the first 100 of them.

<div class="alert alert-danger">Throghout the following exercises you may come across data that are missing. If so, encode these data using a <b>None</b> (Nonetype), unless otherwise stated. </div>

## Retrieving basic track information

In what follows, you are going to retrieve specific data for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 3 </b>Write the code to extract the title (in string form), the name of the album (in string form), the name of the artist (in string form), the duration (in integer form), the track number (in integer form), the popularity (in integer form), and the id (in string form) for all the tracks included in your chosen playlist. Store these data in separate lists called <i>title</i>, <i>album</i>, <i>artist</i>, <i>duration</i>, <i>track_number</i>, <i>track_popularity</i>, and <i>track_id</i>, respectively. In those cases where more than one value is available for these items, retain only the first. In all cases, entries should appear in the same order as they are presented in the playlist.<br><i>[1.25 points]</i></div>

<div class="alert alert-warning">Make sure you correctly set all variable names. They have to be written <b>exactly</b> as given in the instructions.</div>

In [299]:
#Dummy - playlist['items'][0]['track']['artists'][1]

In [300]:
#Test
#playlist['items'][0]['track']
#print(playlist)
#title = playlist['items'][0]['track']['name']
#album = playlist['items'][0]['track']['album']['name']
#artist = playlist['items'][0]['track']['album']['artists'][0]['name']
#duration = int(playlist['items'][0]['track']['duration_ms'])
#track_number = int(playlist['items'][0]['track']['track_number'])
#track_popularity = int(playlist['items'][0]['track']['popularity'])
#track_id = playlist['items'][0]['track']['id']
#print("","Title: ",title,"\n",
#      "Album: ",album,"\n",
#      "Artist: ",artist,"\n",
#      "Song Duration: ",duration,"\n",
#      "Track number: ",track_number,"\n",
#      "Popularity: ",track_popularity,"\n",
#      "ID: ",track_id)

In [301]:
title = []
album = []
artist = []
duration = []
track_number = []
track_popularity = []
track_id = []
for obj in playlist['items']:
    title.append(obj['track']['name'])
    album.append(obj['track']['album']['name'])
    artist.append(obj['track']['artists'][0]['name'])
    duration.append(int(obj['track']['duration_ms']))
    track_number.append(int(obj['track']['track_number']))
    track_popularity.append(int(obj['track']['popularity']))
    track_id.append(obj['track']['id'])

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [302]:
# Variable title exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('title' in locals())
tc.assertIsInstance(title, list)

In [303]:
# Elements in  title exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (str, type(None))) for n in title))

In [304]:
# Elements in title are correct
tc = unittest.TestCase()
tc.assertEqual(title[2], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][2]['track']['name'])
tc.assertEqual(title[5], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][5]['track']['name'])

In [305]:
# Variable title has correct length
tc = unittest.TestCase()
tc.assertEqual(len(title), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [306]:
# Variable album exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('album' in locals())
tc.assertIsInstance(album, list)

In [307]:
# Elements in album exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (str, type(None))) for n in album))

In [308]:
# Elements in album are correct
tc = unittest.TestCase()
tc.assertEqual(album[2], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][2]['track']['album']['name'])
tc.assertEqual(album[5], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][5]['track']['album']['name'])

In [309]:
# Variable album has correct length
tc = unittest.TestCase()
tc.assertEqual(len(album), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [310]:
# Variable artist exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('artist' in locals())
tc.assertIsInstance(artist, list)

In [311]:
# Elements in artist exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (str, type(None))) for n in artist))

In [312]:
# Elements in artist are correct
tc = unittest.TestCase()
tc.assertEqual(artist[2], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][2]['track']['artists'][0]['name'])
tc.assertEqual(artist[5], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][5]['track']['artists'][0]['name'])

In [313]:
# Variable artist has correct length
tc = unittest.TestCase()
tc.assertEqual(len(artist), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [314]:
# Variable duration exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('duration' in locals())
tc.assertIsInstance(duration, list)

In [315]:
# Elements in duration exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (int, type(None))) for n in duration))

In [316]:
# Elements in duration are correct
tc = unittest.TestCase()
tc.assertEqual(duration[2], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][2]['track']['duration_ms'])
tc.assertEqual(duration[5], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][5]['track']['duration_ms'])

In [317]:
# Variable duration has correct length
tc = unittest.TestCase()
tc.assertEqual(len(duration), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [318]:
# Variable track_number exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('track_number' in locals())
tc.assertIsInstance(track_number, list)

In [319]:
# Elements in track_number exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (int, type(None))) for n in track_number))

In [320]:
# Elements in track_number are correct
tc = unittest.TestCase()
tc.assertEqual(track_number[2], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][2]['track']['track_number'])
tc.assertEqual(track_number[5], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][5]['track']['track_number'])

In [321]:
# Variable track_number has correct length
tc = unittest.TestCase()
tc.assertEqual(len(track_number), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [322]:
# Variable track_popularity exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('track_popularity' in locals())
tc.assertIsInstance(track_popularity, list)

In [323]:
# Elements in track_popularity exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (int, type(None))) for n in track_popularity))

In [324]:
# Elements in track_popularity are correct
tc = unittest.TestCase()
tc.assertEqual(track_popularity[2], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][2]['track']['popularity'])
tc.assertEqual(track_popularity[5], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][5]['track']['popularity'])

In [325]:
# Variable track_popularity has correct length
tc = unittest.TestCase()
tc.assertEqual(len(track_popularity), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [326]:
# Variable track_id exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('track_id' in locals())
tc.assertIsInstance(track_id, list)

In [327]:
# Elements in track_id exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (str, type(None))) for n in track_id))

In [328]:
# Elements in track_id are correct
tc = unittest.TestCase()
tc.assertTrue(all('spotify:track:' not in n for n in track_id))
tc.assertEqual(track_id[2], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][2]['track']['id'])
tc.assertEqual(track_id[5], requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][5]['track']['id'])

In [329]:
# Variable track_id has correct length
tc = unittest.TestCase()
tc.assertEqual(len(track_id), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

<div class="alert alert-info"><b>Exercise 4 </b>Write the code to extract the number of available markets (in integer form) for all the tracks included in your chosen playlist. Store these data in a separate list called <i>n_available_markets</i>. As above, entries in this list should appear in the same order as they are presented in the playlist.</div>

In [330]:
available_markets = []
n_available_markets = []
for obj in playlist['items']:
    available_markets = obj['track']['available_markets']
    n_available_markets.append(len(available_markets))

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [331]:
# Variable n_available_markets exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('n_available_markets' in locals())
tc.assertIsInstance(n_available_markets, list)

In [332]:
# Elements in n_available_markets exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (int, type(None))) for n in n_available_markets))

In [333]:
# Elements in n_available_markets are correct
tc = unittest.TestCase()
tc.assertEqual(n_available_markets[2], len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][2]['track']['available_markets']))
tc.assertEqual(n_available_markets[5], len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][5]['track']['available_markets']))

AssertionError: 184 != 183

In [334]:
# Variable n_available_markets has correct length
tc = unittest.TestCase()
tc.assertEqual(len(n_available_markets), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

Each track in the playlist is associated to an album. Let's retrieve the corresponding release date.

<div class="alert alert-info"><b>Exercise 5 </b>Write the code to extract the release year (in int form) and month (in string form) for all the tracks included in your chosen playlist. Store these data in separate lists called <i>release_year</i> and <i>release_month</i>, respectively. Entries should appear in these lists in the same order as they are presented in the playlist.</div>

<div class="alert alert-warning">Make sure you correctly set the variable names. They have to be written <b>exactly</b> as given in the instructions.</div>

<div class="alert alert-warning">Months should be stored with their full names and capitalized: use <i>September</i> for 09, <i>January</i> for 01, etc.</div>

In [335]:
from datetime import date, datetime
import calendar
release_year = []
release_month = []
month = []
release_day = []
d_split = []
for obj in playlist['items']:
    d_splitF = [0,0,0]
    release_d = obj['track']['album']['release_date']
    d_split = release_d.split('-')
    d_splitF = d_split+d_splitF
    release_year.append(int(d_splitF[0]))
    release_month.append(calendar.month_name[int(d_splitF[1])])
    release_day.append(int(d_splitF[2]))

In [336]:
#for obj in playlist['items']:
#    d_splitF = [0,0,0]
#    release_d = obj['track']['album']['release_date']
#    date = datetime.strptime(release_d,'%Y-%m-%d')
#    d_split = release_d.split('-')
#    d_splitF = d_split+d_splitF
#    release_year.append(int(d_splitF[0]))
#    release_month.append(calendar.month_name[int(d_splitF[1])])
#    release_day.append(int(d_splitF[2]))

#from datetime import date, datetime
#import calendar
#release_year = []
#release_month = []
#month = []
#release_day = []
#d_split = []
#for obj in playlist['items']:
#    release_d = obj['track']['album']['release_date']
#    date = datetime.strptime(release_d,'%Y-%m-%d')
#    release_year.append(int(date.strftime("%Y")))
#    release_month.append(date.strftime("%B"))
#    release_day.append(int(date.strftime("%d")))

#month = date.strftime("%B")
#month

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [337]:
# Variable release_year exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('release_year' in locals())
tc.assertIsInstance(release_year, list)

In [338]:
# Elements in release_year exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (int, type(None))) for n in release_year))

In [339]:
__track2 = requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][2]['track']['album']['release_date']
try:
    __date = datetime.strptime(__track2, '%Y-%m-%d')
    __year2 = __date.year
    __month2 = __date.strftime('%B')
except:
    try:
        __date = datetime.strptime(__track2, '%Y-%m')
        __year2 = __date.year
        __month2 = __date.strftime('%B')
    except:
        __year2 = int(__track2)
        __month2 = None

__track5 = requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][5]['track']['album']['release_date']
try:
    __date = datetime.strptime(__track5, '%Y-%m-%d')
    __year5 = __date.year
    __month5 = __date.strftime('%B')
except:
    try:
        __date = datetime.strptime(__track5, '%Y-%m')
        __year5 = __date.year
        __month5 = __date.strftime('%B')
    except:
        __year5 = int(__track5)
        __month5 = None 

# Elements in release_year are correct
tc = unittest.TestCase()     
tc.assertEqual(release_year[2], __year2)
tc.assertEqual(release_year[5], __year5)

In [340]:
# Variable release_year has correct length
tc = unittest.TestCase()
tc.assertEqual(len(release_year), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [341]:
# Variable release_month exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('release_month' in locals())
tc.assertIsInstance(release_month, list)

In [342]:
# Elements in release_month exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (str, type(None))) for n in release_month))

In [343]:
# Elements in release_year are correct
tc = unittest.TestCase()     
tc.assertEqual(release_month[2], __month2)
tc.assertEqual(release_month[5], __month5)

In [344]:
# Variable release_month has correct length
tc = unittest.TestCase()
tc.assertEqual(len(release_month), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

## Retrieving additional track information

In what follows, you are going to retrieve additional data for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 6 </b>Write the code to extract data about the danceability, energy, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence and tempo for all the tracks included in your chosen playlist. Store these data in separate lists called <i>danceability</i> (in float form), <i>energy</i>, <i>loudness</i> (in float form), <i>mode</i> (in int form), <i>speechiness</i> (in float form), <i>acousticness</i> (in float form), <i>instrumentalness</i> (in int and float form), <i>liveness</i> (in float form), <i>valence</i> (in float form) and <i>tempo</i> (in float form), respectively. In those cases where more than one value is available for these items, retain only the first. In all cases, entries should appear in the same order as they are presented in the playlist.</div>

In [345]:
#Check for instrumentalness later

In [346]:
danceability = []
energy = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness_int = []
instrumentalness_float = []
liveness = []
valence = []
tempo = []
playlist_tracksinfo = []
for track in track_id:
    params = {'ids':track}
    response = requests.get(url,params=params,headers=headers).json()
    playlist_tracksinfo.append(response)
    danceability.append(float(response['audio_features'][0]['danceability']))
    energy.append(float(response['audio_features'][0]['energy']))
    loudness.append(float(response['audio_features'][0]['loudness']))
    mode.append(int(response['audio_features'][0]['mode']))
    speechiness.append(float(response['audio_features'][0]['speechiness']))
    acousticness.append(float(response['audio_features'][0]['acousticness']))
    instrumentalness_int.append(int(response['audio_features'][0]['instrumentalness']))
    instrumentalness_float.append(float(response['audio_features'][0]['instrumentalness']))
    liveness.append(float(response['audio_features'][0]['liveness']))
    valence.append(float(response['audio_features'][0]['valence']))
    tempo.append(float(response['audio_features'][0]['tempo']))

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [347]:
# Variable danceability exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('danceability' in locals())
tc.assertIsInstance(danceability, list)

In [348]:
# Elements in danceability exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (float, type(None))) for n in danceability))

In [349]:
# Elements in danceability are correct
tc = unittest.TestCase()
tc.assertEqual(danceability[3], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][3]['track']['id'], headers=headers).json()['danceability'])
tc.assertEqual(danceability[6], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][6]['track']['id'], headers=headers).json()['danceability'])

In [350]:
# Variable danceability has correct length
tc = unittest.TestCase()
tc.assertEqual(len(danceability), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [351]:
# Variable energy exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('energy' in locals())
tc.assertIsInstance(energy, list)

In [352]:
# Elements in energy exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (float, type(None))) for n in energy))

In [353]:
# Elements in energy are correct
tc = unittest.TestCase()
tc.assertEqual(energy[3], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][3]['track']['id'], headers=headers).json()['energy'])
tc.assertEqual(energy[6], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][6]['track']['id'], headers=headers).json()['energy'])

In [354]:
# Variable energy has correct length
tc = unittest.TestCase()
tc.assertEqual(len(energy), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [355]:
# Variable loudness exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('loudness' in locals())
tc.assertIsInstance(loudness, list)

In [356]:
# Elements in loudness exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (float, type(None))) for n in loudness))

In [357]:
# Elements in loudness are correct
tc = unittest.TestCase()
tc.assertEqual(loudness[3], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][3]['track']['id'], headers=headers).json()['loudness'])
tc.assertEqual(loudness[6], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][6]['track']['id'], headers=headers).json()['loudness'])

In [358]:
# Variable loudness has correct length
tc = unittest.TestCase()
tc.assertEqual(len(loudness), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [359]:
# Variable mode exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('mode' in locals())
tc.assertIsInstance(mode, list)

In [360]:
# Elements in mode exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (int, type(None))) for n in mode))

In [361]:
# Elements in mode are correct
tc = unittest.TestCase()
tc.assertEqual(mode[3], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][3]['track']['id'], headers=headers).json()['mode'])
tc.assertEqual(mode[6], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][6]['track']['id'], headers=headers).json()['mode'])

In [362]:
# Variable mode has correct length
tc = unittest.TestCase()
tc.assertEqual(len(mode), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [363]:
# Variable speechiness exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('speechiness' in locals())
tc.assertIsInstance(speechiness, list)

In [364]:
# Elements in speechiness exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (float, type(None))) for n in speechiness))

In [365]:
# Elements in speechiness are correct
tc = unittest.TestCase()
tc.assertEqual(speechiness[3], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][3]['track']['id'], headers=headers).json()['speechiness'])
tc.assertEqual(speechiness[6], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][6]['track']['id'], headers=headers).json()['speechiness'])

In [366]:
# Variable speechiness has correct length
tc = unittest.TestCase()
tc.assertEqual(len(speechiness), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [367]:
# Variable instrumentalness exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('instrumentalness' in locals())
tc.assertIsInstance(instrumentalness, list)

AssertionError: False is not true

In [368]:
# Elements in instrumentalness exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (float, int, type(None))) for n in instrumentalness))

NameError: name 'instrumentalness' is not defined

In [369]:
# Elements in instrumentalness are correct
tc = unittest.TestCase()
tc.assertEqual(instrumentalness[3], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][3]['track']['id'], headers=headers).json()['instrumentalness'])
tc.assertEqual(instrumentalness[6], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][6]['track']['id'], headers=headers).json()['instrumentalness'])

NameError: name 'instrumentalness' is not defined

In [370]:
# Variable instrumentalness has correct length
tc = unittest.TestCase()
tc.assertEqual(len(instrumentalness), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

NameError: name 'instrumentalness' is not defined

In [371]:
# Variable valence exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('valence' in locals())
tc.assertIsInstance(valence, list)

In [372]:
# Elements in valence exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (float, type(None))) for n in valence))

In [373]:
# Elements in valence are correct
tc = unittest.TestCase()
tc.assertEqual(valence[3], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][3]['track']['id'], headers=headers).json()['valence'])
tc.assertEqual(valence[6], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][6]['track']['id'], headers=headers).json()['valence'])

In [374]:
# Variable valence has correct length
tc = unittest.TestCase()
tc.assertEqual(len(valence), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [375]:
# Variable tempo exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('tempo' in locals())
tc.assertIsInstance(tempo, list)

In [376]:
# Elements in tempo exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (float, type(None))) for n in tempo))

In [377]:
# Elements in tempo are correct
tc = unittest.TestCase()
tc.assertEqual(tempo[3], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][3]['track']['id'], headers=headers).json()['tempo'])
tc.assertEqual(tempo[6], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][6]['track']['id'], headers=headers).json()['tempo'])

In [378]:
# Variable tempo has correct length
tc = unittest.TestCase()
tc.assertEqual(len(tempo), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [379]:
# Variable acousticness exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('acousticness' in locals())
tc.assertIsInstance(acousticness, list)

In [380]:
# Elements in acousticness exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (float, type(None))) for n in acousticness))

In [381]:
# Elements in acousticness are correct
tc = unittest.TestCase()
tc.assertEqual(acousticness[3], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][3]['track']['id'], headers=headers).json()['acousticness'])
tc.assertEqual(acousticness[6], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][6]['track']['id'], headers=headers).json()['acousticness'])

In [382]:
# Variable acousticness has correct length
tc = unittest.TestCase()
tc.assertEqual(len(acousticness), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [383]:
# Variable liveness exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('liveness' in locals())
tc.assertIsInstance(liveness, list)

In [384]:
# Elements in liveness exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (float, type(None))) for n in liveness))

In [385]:
# Elements in liveness are correct
tc = unittest.TestCase()
tc.assertEqual(liveness[3], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][3]['track']['id'], headers=headers).json()['liveness'])
tc.assertEqual(liveness[6], requests.get(base_url + 'audio-features/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][6]['track']['id'], headers=headers).json()['liveness'])

In [386]:
# Variable liveness has correct length
tc = unittest.TestCase()
tc.assertEqual(len(liveness), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

## Retrieving artist information

In what follows, you are going to retrieve data about the artists for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 7 </b>Write the code to extract data about the total number of followers (in int form), the first listed genre (in string form) and the popularity (in int form) for the artists of all the tracks included in your chosen playlist. Store these data in separate lists called <i>artist_followers</i>, <i>genre</i> and <i>artist_popularity</i>. In cases where no genre is given, fill in the corresponding entry using a None.</div>

<div class="alert alert-warning">Note that the artist information has to to be extracted from the tracks themselves.</div>

In [387]:
#For Later - check for none, clarify how, with Harsha?

In [388]:
artists_id = []
artist_followers = []
genre = []
artist_popularity = []
all_art = []
for obj in playlist['items']:
    artistID = obj['track']['artists'][0]['uri'].split(':')[2]
    artists_id.append(artistID)
    params = {'ids':artistID}
    art = requests.get(base_url+'artists/',params=params,headers=headers).json()
    genre.append(art['artists'][0]['genres'])
#    if(art['artists'][0]['genres']==[]):
#        genre.append(None)      
#    else:
#        genre.append(art['artists'][0]['genres'])
    artist_popularity.append(art['artists'][0]['popularity'])
    all_art.append(art)
    artist_follow = art['artists'][0]['followers']['total']
    artist_followers.append(int(artist_follow))

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [389]:
# Variable artist_followers exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('artist_followers' in locals())
tc.assertIsInstance(artist_followers, list)

In [390]:
# Elements in artist_followers exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (int, type(None))) for n in artist_followers))

In [391]:
# Elements in artist_followers are correct
tc = unittest.TestCase()
tc.assertEqual(artist_followers[1], requests.get(base_url + 'artists/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][1]['track']['artists'][0]['id'], headers=headers).json()['followers']['total'])
tc.assertEqual(artist_followers[4], requests.get(base_url + 'artists/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][4]['track']['artists'][0]['id'], headers=headers).json()['followers']['total'])

In [392]:
# Variable artist_followers has correct length
tc = unittest.TestCase()
tc.assertEqual(len(artist_followers), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [393]:
# Variable genre exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('genre' in locals())
tc.assertIsInstance(genre, list)

In [394]:
# Elements in genre exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (str, type(None))) for n in genre))

AssertionError: False is not true

In [395]:
# Elements in genre are correct
output = requests.get(base_url + 'artists/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][2]['track']['artists'][0]['id'], headers=headers).json()['genres']

try:
    value = output[0]
except:
    value = None

tc = unittest.TestCase()
tc.assertEqual(genre[2], value)

output = requests.get(base_url + 'artists/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][4]['track']['artists'][0]['id'], headers=headers).json()['genres']

try:
    value = output[0]
except:
    value = None

tc = unittest.TestCase()
tc.assertEqual(genre[4], value)

AssertionError: ['electronica', 'folktronica', 'intelligent dance music'] != 'electronica'

In [397]:
# Variable genre has correct length
tc = unittest.TestCase()
tc.assertEqual(len(genre), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [398]:
# Variable artist_popularity exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('artist_popularity' in locals())
tc.assertIsInstance(artist_popularity, list)

In [399]:
# Elements in artist_popularity exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (int, type(None))) for n in artist_popularity))

In [400]:
# Elements in artist_popularity are correct
tc = unittest.TestCase()
tc.assertEqual(artist_popularity[1], requests.get(base_url + 'artists/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][1]['track']['artists'][0]['id'], headers=headers).json()['popularity'])
tc.assertEqual(artist_popularity[4], requests.get(base_url + 'artists/' + requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items'][4]['track']['artists'][0]['id'], headers=headers).json()['popularity'])

In [401]:
# Variable artist_popularity has correct length
tc = unittest.TestCase()
tc.assertEqual(len(artist_popularity), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In addition to the above, there's also additional information we can retrieve about each artist. For this purpose, let's first retrieve the list of distinct artists for the different tracks in your playlist.

<div class="alert alert-info"><b>Exercise 8 </b>Write the code to identify the list of unique artist ids that correspond to the different tracks in your chosen playlist. Store these data in a list called <i>unique_artist_ids</i>.</div>

<div class="alert alert-warning">Retrieve the artist ids that correspod directly to the tracks.</div>

In [402]:
unique_artist_ids=list(set(artists_id))

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [403]:
# Variable unique_artist_ids exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('unique_artist_ids' in locals())
tc.assertIsInstance(unique_artist_ids, list)

In [404]:
# Elements in unique_artist_ids exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (str)) for n in unique_artist_ids))

In [405]:
__playlist = requests.get(base_url + 'playlists/'+playlist_id+'/tracks', 
                                 headers=headers).json()
# Variable unique_artist_ids has right length
tc = unittest.TestCase()
tc.assertEqual(len(set([item['track']['artists'][0]['id'] for item in __playlist['items']])), len(unique_artist_ids))

In [406]:
# Variable unique_artist_ids has right content
tc = unittest.TestCase()
tc.assertTrue(all(elem in unique_artist_ids for elem in set([item['track']['artists'][0]['id'] for item in __playlist['items']])))

We are now interested in retrieving catalog information about each artist’s top tracks. This information is provided by Spotify's API on a country basis. Here, we will retrieve the information corresponding to Spain, whose *ISO 3166-1 alpha-2* code is **ES**. The information we are looking for is stored under the ```top-tracks``` endpoint for ```artists```. Requests to this location retrieve the 10 most famous tracks for a given artist id.

<div class="alert alert-info"><b>Exercise 9 </b>Write the code to retrieve the 10 top tracks for each of the unique artists in your chosen playlist. Store these data in a dictionary called <i>top_tracks</i>. The keys of this dictionary should correspond to the unique artist ids stored in the list <i>unique_artist_id</i>. The values of this dictionary should include the names of the 10 most popular songs in a list.</div>

<div class="alert alert-warning">In cases where the number of tracks included in the top-tracks endpoint is below 10, simply retrieve the provided list.</div>


---

In [424]:
#params = {'ids': str(unique_artists_ids[0]),'country':'ES'}
#top_tracks = requests.get(base_url+"artists/"+str(unique_artists_ids)+"/top-tracks",headers=headers,country=).json()
#top_tracks

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [ ]:
# Variable top_tracks exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('top_tracks' in locals())
tc.assertIsInstance(top_tracks, dict)

In [ ]:
# Keys of top_tracks correspond to unique artist ids
tc = unittest.TestCase()
tc.assertEqual(top_tracks.keys(), set([item['track']['artists'][0]['id'] for item in __playlist['items']]))

In [ ]:
# Values of top_tracks correspond to lists of 10 elements
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(value, list) for value in top_tracks.values()))
tc.assertTrue(all(len(value)<=10 for value in top_tracks.values()))

We can now use this information to identify those songs in your chosen playlist that correspond to each artist's top tracks.

<div class="alert alert-info"><b>Exercise 10 </b>Write the code to check whether the different tracks in your chosen playlist are included among the corresponding artist's top tracks. Store the results in a list called <i>is_top</i>. This list should include the boolean value <i>True</i> whenever the considered track is among the top tracks for that artist and <i>False</i> otherwise.</div>

<div class="alert alert-warning">When comparing track names, consider only <b>exact</b> matches.</div>

In [ ]:
# YOUR CODE HERE

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [ ]:
# Variable is_top exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('is_top' in locals())
tc.assertIsInstance(is_top, list)

In [ ]:
# Elements in is_top exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (bool)) for n in is_top))

In [ ]:
# Variable is_top has right length
tc = unittest.TestCase()
tc.assertEqual(len(is_top), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [ ]:
artist_id = __playlist['items'][2]['track']['artists'][0]['id']
track_name = __playlist['items'][2]['track']['name']

# Variable is_top has right content
tc = unittest.TestCase()
tc.assertEqual(is_top[2], True if track_name in top_tracks[artist_id] else False)

artist_id = playlist['items'][5]['track']['artists'][0]['id']
track_name = playlist['items'][5]['track']['name']

# Variable is_top has right content
tc = unittest.TestCase()
tc.assertEqual(is_top[5], True if track_name in top_tracks[artist_id] else False)

A very interesting feature in Spotify is the song recommender.

When creating a playlist from scratch, Spotify offers help by suggesting additional tracks that may be related to the ones already added. Spotify's API also provides a specific endpoint for `recommendations`. We will use this endpoint to retrieve one recommendation for each song in our original playlist.

<div class="alert alert-info"><b>Exercise 11 </b>Write the code to retrieve the id, title and artist from the list of recommendations for each track in your playlist. Store these data in new lists called <i>recommendatin_id</i>, <i>recommendation_title</i> and <i>recommendation_artist</i>.</div>

<div class="alert alert-warning">Retrieve recommendations based solely on each track.</div>

<div class="alert alert-warning">Retrieve information only about the first track in the recommendations list.</div>

<div class="alert alert-warning">When more than one artist is associated to the recommended track, retrieve information only about the first.</div>

In [ ]:
# YOUR CODE HERE

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [ ]:
# Variable recommendation_id exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('recommendation_id' in locals())
tc.assertIsInstance(recommendation_id, list)

In [ ]:
# Variable recommendation_id has the right length
tc = unittest.TestCase()
tc.assertTrue(len(recommendation_id)==len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [ ]:
# Elements in recommendation_id are correct
tc = unittest.TestCase()
tc.assertEqual(recommendation_id[1], requests.get(base_url + 'recommendations',params={'seed_tracks': [track_id[1]]} , headers=headers).json()['tracks'][0]['id'])
tc.assertEqual(recommendation_id[7], requests.get(base_url + 'recommendations',params={'seed_tracks': [track_id[7]]} , headers=headers).json()['tracks'][0]['id'])

In [ ]:
# Variable recommendation_title exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('recommendation_title' in locals())
tc.assertIsInstance(recommendation_title, list)

In [ ]:
# Variable recommendation_title has the right length
tc = unittest.TestCase()
tc.assertTrue(len(recommendation_title)==len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [ ]:
# Elements in recommendation_title are correct
tc = unittest.TestCase()
tc.assertEqual(recommendation_title[1], requests.get(base_url + 'recommendations',params={'seed_tracks': [track_id[1]]} , headers=headers).json()['tracks'][0]['name'])
tc.assertEqual(recommendation_title[7], requests.get(base_url + 'recommendations',params={'seed_tracks': [track_id[7]]} , headers=headers).json()['tracks'][0]['name'])

In [ ]:
# Variable recommendation_artist exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('recommendation_artist' in locals())
tc.assertIsInstance(recommendation_artist, list)

In [ ]:
# Variable recommendation_artist has the right length
tc = unittest.TestCase()
tc.assertTrue(len(recommendation_artist)==len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [ ]:
# Elements in recommnedation_artist are correct
tc = unittest.TestCase()
tc.assertEqual(recommendation_artist[1], requests.get(base_url + 'recommendations',params={'seed_tracks': [track_id[1]]} , headers=headers).json()['tracks'][0]['artists'][0]['name'])
tc.assertEqual(recommendation_artist[7], requests.get(base_url + 'recommendations',params={'seed_tracks': [track_id[7]]} , headers=headers).json()['tracks'][0]['artists'][0]['name'])

The final step is to save the data you have collected to a csv file. You can do so by first saving the data to a pandas DataFrame and then exporting it to a csv file of your choosing, in the same directory where your notebook is located.

## Saving the data

In what follows, you are going to store all the data you just retrieved in a convenient form.

<div class="alert alert-info"><b>Exercise 12 </b>Write the code to save the data about the title, the name of the album, the name of the artist, the duration, the track number, the release date, the popularity, the id, the number of available markets, the danceability, the energy, the loudness, the mode, the speechiness, the acousticness, the instrumentalness, the liveness, the valence and the tempo for all the tracks included in your chosen playlist, as well as the data about the total number of followers, the first listed genre, the popularity for the artists, whether the tracks are included in the top 10 and what the id, the title and the artist of the first song in the recommendations list are in a dataframe called <i>df</i> and save this DataFrame to a .csv file called 'spotify.csv'. When creating the dataframe make sure the column names are <b>exactly</b> the same as those of the lists you created in previous exercises to store the different values.</div>

In [ ]:
# YOUR CODE HERE

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [ ]:
import pandas as pd

# Variable idf exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('df' in locals())
tc.assertIsInstance(df, pd.DataFrame)

In [ ]:
# Variable df has right number of columns
tc = unittest.TestCase()
tc.assertTrue(len(df.columns)==26)

In [ ]:
# Variable df has right column names
tc = unittest.TestCase()
tc.assertTrue(all(col in ['acousticness', 'album', 'artist', 'artist_followers',
       'artist_popularity', 'danceability', 'duration', 'energy',
       'instrumentalness', 'genre', 'liveness', 'loudness', 'mode',
       'n_available_markets', 'release_date', 'speechiness', 'tempo', 'title', 'track_id',
       'track_number', 'track_popularity', 'valence', 'is_top', 'recommendation_id',
       'recommendation_title', 'recommendation_artist'] for col in df.columns))

In [ ]:
# Variable df has shape
tc = unittest.TestCase()
tc.assertTrue(len(df)==len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

## Bonus exercise

You can complete the exercises below to obtain extra points. These points will be added to your score directly. The maximum score in this assignment is a 10.

<div class="alert alert-danger"><b>Bonus 1 </b>Write the code to find the most popular song in your playlist. If several songs have the same popularity, choose the one for which the artist has the most followers. Save the name of the song to a new variable called <i>most_popular</i>. This variable should <b>only</b> contain the name of the most popular song in string form.</div>

In [ ]:
# YOUR CODE HERE

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [ ]:
# Variable most_popular exists and is of right type has right length
tc = unittest.TestCase()
tc.assertTrue('most_popular' in locals())
tc.assertIsInstance(most_popular, str)

In [ ]:
__track_popularity = []
__artist_followers = []
for item in __playlist['items']:
    __artist_id = item['track']['artists'][0]['id']
    __track_popularity.append(item['track']['popularity'])
    response = requests.get(base_url + 'artists/' + __artist_id, headers=headers).json()
    __artist_followers.append(response['followers']['total'])
__df = pd.DataFrame({'track_popularity': track_popularity, 
                     'artist_followers': artist_followers,
                     'title': title})
# Variable most_popular includes right value has right length

__df_ = __df[__df['track_popularity']==__df['track_popularity'].max()] 
__most_popular = __df_[__df_['artist_followers']==__df_['artist_followers'].max()]['title'].item()

tc = unittest.TestCase()
tc.assertEqual(most_popular, __most_popular)

You might have noticed that there exist slight incosistencies in the way in which the track title are included in the playlist, versus the top 10 artist tracks. For example, the fourht song in the provided playlist appears as <i>High And Dry</i> within the title of the playlist songs but as <i>High and Dry</i> (with a small "a" for the word "and") in the Radiohead's top tracks. This results in the fact that the song of the playlist will not be included as one of the artist's top tracks in the is_top list.

<div class="alert alert-danger"><b>Bonus 2 </b>Write the code to create a new list called <i>is_top_2</i>. This time make sure that you look for partial matches of title names too.</div>

In [ ]:
# YOUR CODE HERE

The following cells check whether your code is correct. Please **don't write any code here**. Just leave them as they are.

In [ ]:
# Variable is_top_2 exists and is of correct type
tc = unittest.TestCase()
tc.assertTrue('is_top_2' in locals())
tc.assertIsInstance(is_top_2, list)

In [ ]:
# Elements in is_top_2 exists are of correct type
tc = unittest.TestCase()
tc.assertTrue(all(isinstance(n, (str)) for n in is_top_2))

In [ ]:
# Variable is_top_2 has right length  
tc = unittest.TestCase()
tc.assertEqual(len(is_top_2), len(requests.get(base_url + 'playlists/'+playlist_id+'/tracks', headers=headers).json()['items']))

In [ ]:
artist_id = __playlist['items'][2]['track']['artists'][0]['id']
track_name = __playlist['items'][2]['track']['name']

# Variable is_top_2 has right content
tc = unittest.TestCase()
tc.assertEqual(is_top_2[2], 'yes' if track_name.lower() in ",".join(top_tracks[artist_id]).lower() else 'no')

artist_id = playlist['items'][5]['track']['artists'][0]['id']
track_name = playlist['items'][5]['track']['name']

# Variable is_top has right content
tc = unittest.TestCase()
tc.assertEqual(is_top_2[5], 'yes' if track_name.lower() in ",".join(top_tracks[artist_id]).lower() else 'no')